# ML Pipeline Preparation
In this section the ML pipeline will be made.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine

import re
from sklearn.pipeline import Pipeline

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)
#df.head()

In [3]:
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message']  # Message Column
Y = df.iloc[:, 4:] # Classification label

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    #Normalize words to lowercase
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #Tokenize the text
    token_word = word_tokenize(text)
    
    #Use this to group together similar words
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in token_word:
        clean_token = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_token)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())) #Random Forest Used
])

### 4. Train pipeline

In [7]:
#Split data into test and train data sets
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [8]:
#Train Pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [10]:
#Function to iterate through columns and call sklearn classification report on each.
def test_model(y_test, y_pred):
    for index, column in enumerate(y_test):
        print(column, classification_report(y_test[column], y_pred[:, index]))

In [11]:
test_model(y_test, y_pred)

related              precision    recall  f1-score   support

          0       0.61      0.35      0.44      1540
          1       0.82      0.93      0.87      4970
          2       0.48      0.50      0.49        44

avg / total       0.77      0.79      0.77      6554

request              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5453
          1       0.83      0.40      0.54      1101

avg / total       0.88      0.89      0.87      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6533
          1       0.00      0.00      0.00        21

avg / total       0.99      1.00      1.00      6554

aid_related              precision    recall  f1-score   support

          0       0.74      0.86      0.80      3876
          1       0.74      0.56      0.63      2678

avg / total       0.74      0.74      0.73      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
#Test for accuracy
accuracy = (y_pred == y_test).mean()
accuracy

related                   0.789289
request                   0.885566
offer                     0.996796
aid_related               0.738175
medical_help              0.921117
medical_products          0.951633
search_and_rescue         0.972536
security                  0.980623
military                  0.969942
child_alone               1.000000
water                     0.946750
food                      0.917608
shelter                   0.925389
clothing                  0.987031
money                     0.975893
missing_people            0.988557
refugees                  0.965212
death                     0.955447
other_aid                 0.875954
infrastructure_related    0.933781
transport                 0.954074
buildings                 0.951938
electricity               0.981843
tools                     0.993744
hospitals                 0.990388
shops                     0.994202
aid_centers               0.988404
other_infrastructure      0.954226
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fe78a3c6f28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [14]:
#Create grid search parameters
parameters = {
    'clf__estimator__n_estimators': [1, 25]
    
}

#Grid search
cv = GridSearchCV(pipeline, param_grid=parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [1, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
cv.fit(X_train, y_train)
cv.best_params_

{'clf__estimator__n_estimators': 25}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
y_pred = cv.predict(X_test)
test_model(y_test, y_pred)

related              precision    recall  f1-score   support

          0       0.69      0.29      0.41      1540
          1       0.81      0.96      0.88      4970
          2       0.39      0.32      0.35        44

avg / total       0.78      0.80      0.76      6554

request              precision    recall  f1-score   support

          0       0.90      0.98      0.94      5453
          1       0.82      0.44      0.57      1101

avg / total       0.88      0.89      0.88      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6533
          1       0.00      0.00      0.00        21

avg / total       0.99      1.00      1.00      6554

aid_related              precision    recall  f1-score   support

          0       0.77      0.85      0.81      3876
          1       0.75      0.64      0.69      2678

avg / total       0.76      0.77      0.76      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
#Test for accuracy
accuracy = (y_pred == y_test).mean()
accuracy

related                   0.796613
request                   0.890296
offer                     0.996796
aid_related               0.766250
medical_help              0.924168
medical_products          0.950870
search_and_rescue         0.973451
security                  0.980775
military                  0.968569
child_alone               1.000000
water                     0.953464
food                      0.925236
shelter                   0.928593
clothing                  0.985810
money                     0.975893
missing_people            0.988709
refugees                  0.966128
death                     0.956668
other_aid                 0.877174
infrastructure_related    0.934696
transport                 0.956668
buildings                 0.954684
electricity               0.982453
tools                     0.993744
hospitals                 0.990540
shops                     0.994202
aid_centers               0.988404
other_infrastructure      0.954074
weather_related     

### 8. The model can be improved further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [18]:
import pickle
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.